### Step 11: Calculate nuclear fraction with DropletQC

Now we'll reanalyze the human single-nucleus RNA-seq data shown in Lee et al's Figure 5, which is a combination of two previously published datasets from [Absinta et al 2021 (PMID: 34497421)](https://pubmed.ncbi.nlm.nih.gov/34497421/) and [Schirmer et al 2019 (PMID: 31316211)](https://pubmed.ncbi.nlm.nih.gov/31316211/).

Prior to our main reanalysis in Steps 12 & 13, we will calculate the nuclear fraction statistic for each cell in these datasets. 

Nuclear fraction is a quality control metric established by the tool DropletQC which reflects the proportion of RNA in a given cell which is unspliced, pre-mRNA ([Muskovic & Powell 2021 (PMID: 34857027)(https://pubmed.ncbi.nlm.nih.gov/34857027/)]). In single-nucleus RNA-seq data, a lower nuclear fraction may be a sign that a given droplet did not contain a nucleus, and is thus an empty droplet mis-called as cell containing due to the prescence of ambient RNA or cytoplasm contamination ([Montserrat-Ayuso & Esteve-Codina 2024 (PMID: 39574015)(https://pubmed.ncbi.nlm.nih.gov/39574015/)]).

Because DropletQC requires alignment BAM files as input for calculating nuclear fraction, we first retrieve the FASTQ files for the Absinta and Schirmer datasets from the Sequence Read Archive (SRA) repository. These datasets are present at accessions [GSE180759](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE180759) and [PRJNA544731](https://www.ncbi.nlm.nih.gov/bioproject/PRJNA544731). Then, we preprocess and align the sequence reads using [Cell Ranger](https://www.10xgenomics.com/support/software/cell-ranger/latest/release-notes/cr-release-notes#v7-0-1).

These steps were performed on a Linux HPC cluster using the SLURM resource manager using NCBI's [sra-tools program](https://github.com/ncbi/sra-tools). These bash scripts are shown below, but are not run in this notebook due to the large file sizes and run times.

All HPC scripts can be found in the `code/HPC_scripts` folder.

Note that HPC scripts have some file paths partially censored as `...`. Running these scripts on your own system will require updating these file paths and changing SLURM parameters to match your HPC system.

The SRA download scripts utilize the `sra-downloads` conda environment, which can be created from the `sra-downloads.yml` file in the `code/environments` directory.

The below script downloads the Absinta dataset FASTQ files from SRA.

It requires as input the `Absinta_SRR_Acc_List.txt` file, which contains a list of sample accession numbers. This file can be found in the `inputs/absinta` folder.

```
#!/bin/bash
#SBATCH --partition=cpu_long,fn_long # partition on which to run
#SBATCH --job-name=sra_absinta # name
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL)
#SBATCH --mail-user=michael.odea@nyulangone.org #Where to send mail
#SBATCH --ntasks=6 #Run on # of CPUs
#SBATCH --mem-per-cpu=8gb #Job memory request ##max 128GB
#SBATCH --time=28-00:00:00 #Time limit hrs:min:sec
#SBATCH --output=./sra_downloads_%a.log #Standard output and error log
#SBATCH --no-kill
#SBATCH --array=0-37

source /.../anaconda3/bin/activate sra-downloads

cd ./inputs/absinta/fastq

mapfile -t samples < ../Absinta_SRR_Acc_List.txt # change the file path to this input file as needed

vdb-config --prefetch-to-cwd

prefetch --max-size u ${samples[$SLURM_ARRAY_TASK_ID]}

vdb-validate ${samples[$SLURM_ARRAY_TASK_ID]}

fasterq-dump -e 6 ${samples[$SLURM_ARRAY_TASK_ID]}

gzip "${samples[$SLURM_ARRAY_TASK_ID]}"_*".fastq"
```

This next script downloads the Schirmer dataset FASTQ files from SRA. It requires as input the `Schirmer_SRR_Acc_List.txt` file, which contains a list of sample accession numbers. This file can be found in the `inputs/schirmer` folder.

```
#!/bin/bash
#SBATCH --partition=cpu_long,fn_long # partition on which to run
#SBATCH --job-name=sra_schirmer # name
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL)
#SBATCH --ntasks=6 #Run on # of CPUs
#SBATCH --mem-per-cpu=8gb #Job memory request ##max 128GB
#SBATCH --time=28-00:00:00 #Time limit hrs:min:sec
#SBATCH --output=./sra_downloads.log #Standard output and error log
#SBATCH --no-kill
#SBATCH --array=0-20

source /.../anaconda3/bin/activate sra-downloads

cd ./inputs/schirmer/fastq

mapfile -t samples < ../Schirmer_SRR_Acc_List.txt # change the file path to this input file as needed

vdb-config --prefetch-to-cwd

prefetch --max-size u ${samples[$SLURM_ARRAY_TASK_ID]}

vdb-validate ${samples[$SLURM_ARRAY_TASK_ID]}

fasterq-dump -e 6 ${samples[$SLURM_ARRAY_TASK_ID]}

gzip "${samples[$SLURM_ARRAY_TASK_ID]}"_*".fastq"
```

Next, we process the FASTQ files with [Cell Ranger (version 7.0.1)](https://www.10xgenomics.com/support/software/cell-ranger/latest/release-notes/cr-release-notes#v7-0-1). Because the names of the FASTQ files downloaded from SRA did not match the format expected by Cell Ranger, we first ran the following bash script to rename them:

```
#!/bin/bash

### name: rename_fastqs.sh
### usage: bash rename_fastqs.sh

# Directory containing the .fastq.gz files
directory=./inputs/absinta/fastq # change the input directory file path as needed
# we also ran this script for the schirmer fastq files
# directory=./inputs/schirmer/fastq

# Change to the target directory
cd "$directory" || { echo "Directory not found: $directory"; exit 1; }

# Iterate over the .fastq.gz files
for file in *.fastq.gz; do
  # Check if the file matches the pattern SRRXXXXXXX_1.fastq.gz or SRRXXXXXXX_2.fastq.gz
  if [[ $file =~ ^(SRR[0-9]+)_(1|2)\.fastq\.gz$ ]]; then
    # Extract the base name and the read number
    base="${BASH_REMATCH[1]}"
    read_num="${BASH_REMATCH[2]}"
    
    # Determine the new read number and file suffix
    if [ "$read_num" -eq 1 ]; then
      new_file="${base}_S1_R1_001.fastq.gz"
    elif [ "$read_num" -eq 2 ]; then
      new_file="${base}_S1_R2_001.fastq.gz"
    else
      # Skip files that do not match the expected pattern
      continue
    fi
    
    # Rename the file
    mv "$file" "$new_file"
    echo "Renamed $file to $new_file"
  fi
done
```

We then run Cell Ranger on the Absinta dataset. This script requires you have a working Cell Ranger version 7.0.1 installed.

This script requires the `Absinta_SraRunTable.txt` file as input (which was downloaded from SRA). This file can be found in the `inputs/absinta` folder.

This script also requires the pre-built Cell Ranger genome reference package `refdata-gex-GRCh38-2024-A` (Human reference (GRCh38) - 2024-A), which can be downloaded from the [10X Genomics website](https://www.10xgenomics.com/support/software/cell-ranger/downloads#reference-downloads). 

```
#!/bin/bash
#SBATCH --partition=cpu_medium # partition on which to run
#SBATCH --job-name=absinta_cr_count # name
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL)
#SBATCH --ntasks=1 #Run on # of CPUs
#SBATCH --cpus-per-task=16
#SBATCH --mem=128gb #Job memory request ##max 128GB
#SBATCH --time=3-00:00:00 #Time limit hrs:min:sec
#SBATCH --output=./cellranger_count_%a.log #Standard output and error log
#SBATCH --no-kill
#SBATCH --array=0-19

cd ./inputs/absinta/CellRanger

cat ../Absinta_SraRunTable.txt | cut -d',' -f32 | uniq | grep "GSM" > absinta_samples.txt

mapfile -t samples < absinta_samples.txt

sample_ids=$(grep ${samples[$SLURM_ARRAY_TASK_ID]} ../Absinta_SraRunTable.txt | cut -d',' -f1 | tr '\n' ',' | sed 's/\(.*\),/\1 /')

# id		 name for output directory corresponding to sample
# fastqs         Path of folder created by 10x demultiplexing or bcl2fastq
# transcriptome  Path of folder containing 10X-compatible transcriptome

/.../CellRanger/cellranger-7.0.1/cellranger count \
--id "${samples[$SLURM_ARRAY_TASK_ID]}" \
--localmem 128 \
--localcores 16 \
--transcriptome=/.../CellRanger/references/refdata-gex-GRCh38-2024-A \
--fastqs ../fastq \
--include-introns=true \
--sample $sample_ids
```

We then run Cell Ranger on the Schirmer dataset using a similar script:

```
#!/bin/bash
#SBATCH --partition=cpu_medium # partition on which to run
#SBATCH --job-name=schirmer_cr_count # name
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL)
#SBATCH --ntasks=1 #Run on # of CPUs
#SBATCH --cpus-per-task=16
#SBATCH --mem=128gb #Job memory request ##max 128GB
#SBATCH --time=3-00:00:00 #Time limit hrs:min:sec
#SBATCH --output=./cellranger_count_%a.log #Standard output and error log
#SBATCH --no-kill
#SBATCH --array=0-20

cd ./inputs/schirmer/CellRanger

cat ../Schirmer_SraRunTable.txt | cut -d',' -f35 | uniq | grep -P '\S' > schirmer_samples.txt

mapfile -t samples < schirmer_samples.txt

sample_ids=$(grep ${samples[$SLURM_ARRAY_TASK_ID]} ../Schirmer_SraRunTable.txt | cut -d',' -f1 | tr '\n' ',' | sed 's/\(.*\),/\1 /')

# id		 name for output directory corresponding to sample
# fastqs         Path of folder created by 10x demultiplexing or bcl2fastq
# transcriptome  Path of folder containing 10X-compatible transcriptome

/.../CellRanger/cellranger-7.0.1/cellranger count \
--id "${samples[$SLURM_ARRAY_TASK_ID]}" \
--localmem 128 \
--localcores 16 \
--transcriptome=/.../CellRanger/references/refdata-gex-GRCh38-2024-A \
--fastqs ../fastq \
--include-introns=true \
--sample $sample_ids
```

Next we run DropletQC for the Absinta and Schirmer datasets.

In [1]:
library(DropletQC)
library(data.table)
library(ggplot2)
library(dplyr)
library(Seurat)
setwd('..') # changing working directory to 'EpiMemAstros' folder. Adjust as needed.


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t




Read in a CSV file which contains the SRA sample information for the Absinta dataset samples:

In [2]:
absinta_geo_key = read.table('inputs/zenodo/absinta_GEO_key.csv', sep=',', header = TRUE)
absinta_geo_key

GEO_ident,Sample,NBB,Block,Diagnosis,Tissue,Dataset
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM5470483,3,13_047,MS4-2,MS,Chronic active edge,Absinta
GSM5470484,11,09_034,MS1-2,MS,Chronic inactive edge,Absinta
GSM5470485,4,12_078,MS2-1,MS,Chronic active edge,Absinta
GSM5470486,5,09_067,MS3-1,MS,Chronic active edge,Absinta
GSM5470487,6,13_047,MS4-2,MS,Chronic active edge,Absinta
GSM5470488,12,12_078,MS2-2,MS,Chronic inactive edge,Absinta
GSM5470489,15,11_069,CTRL2,CTRL,Normal frontal white matter,Absinta
GSM5470490,16,13_047,MS4-1,MS,Lesion core,Absinta
GSM5470491,17,09_034,MS1-1,MS,Periplaque,Absinta


Now, calculate the nuclear fraction for all barcodes in the Absinta dataset using DropletQC. This script requires the `possorted_genome_bam.bam` and `possorted_genome_bam.bam.bai` files, as well as the `raw_feature_bc_matrix/barcodes.tsv.gz` file, produced by CellRanger for each sample. Because these files are very large (requiring several hundred GBs of disk space), and because the run time for the step is quite long, this code block is currently commented out (with `#` preceding each line). We have provided .rds files in the Zenodo repository which contain the nuclear fractions we calculated, and we will use these files in the Steps 12 & 13 scripts instead.

If you wish to obtain the BAM, BAM index, and barcodes files produced from our Cell Ranger run, please email `michael.odea@nyulangone.org` and `shane.liddelow@nyulangone.org`. We will be happy to arrange transfer of these files to you.

If you have obtained the input files and wish to run DropletQC yourself, remove the "#" on each line in the below cell to un-comment the code. You can do this by selecting the lines and entering `Ctrl + /` or `Cmd + /` (depending on your platform OS).

In [3]:
#absinta_nf_df <- c()
#
#for(x in 1:20){
#
#    geo_ident = absinta_geo_key %>% filter(Sample == x) %>% .$GEO_ident
#   
#    print(paste0("Processing sample ", geo_ident, "..."))
#    
#    nf_tmp = nuclear_fraction_tags(bam = paste0("inputs/absinta/CellRanger/", geo_ident, "/outs/possorted_genome_bam.bam"), 
#                     bam_index = paste0("inputs/absinta/CellRanger/", geo_ident, "/outs/possorted_genome_bam.bam.bai"),
#                     barcodes = paste0("inputs/absinta/CellRanger/", geo_ident, "/outs/raw_feature_bc_matrix/barcodes.tsv.gz"))
#
#    rownames(nf_tmp) <- paste0(gsub("(.*)-(.*)", "\\1", rownames(nf_tmp)), "-", x)
#
#    if(x == 1){
#        absinta_nf_df <- nf_tmp
#    } else{
#        absinta_nf_df <- rbind(absinta_nf_df, nf_tmp)
#    }
#}
#
#print(dim(absinta_nf_df))
#      
#saveRDS(absinta_nf_df, "outputs/absinta_nuclear_fraction_df.rds")

We'll now repeat the process for the Schirmer dataset.

To do this, we first read in a preprocessed Seurat object of the combined Absinta and Schirmer datasets provided to us by the authors of Lee et al, which we will use to extract sample metadata.

First load the Seurat object:

In [4]:
human <- readRDS('inputs/zenodo/Human.MS.Astrocytes.with.memory.astrocytes.marked.out.rds')
# convert a v3 assay to a v5 assay
human[["RNA"]] <- as(object = human[["RNA"]], Class = "Assay5")
human[["RNA"]]$data <- as(object = human[["RNA"]]$data, Class = "dgCMatrix")

human

Warning message:
“Assay RNA changing from Assay to Assay5”


An object of class Seurat 
91241 features across 16276 samples within 2 assays 
Active assay: RNA (61992 features, 0 variable features)
 2 layers present: counts, data
 1 other assay present: SCT
 3 dimensional reductions calculated: pca, harmony, umap

Now split the Seurat object into its component datasets, and extract the Schirmer sample metadata.

In [5]:
schirmer = human[,WhichCells(human, expression = `DataSource` == "Schirmer_2019")]
absinta = human[,setdiff(colnames(human), WhichCells(human, expression = `DataSource` == "Schirmer_2019"))]
schirmer_annotation = schirmer@meta.data[, c('sample', 'Original_sample_name', 'pathology')]
schirmer_key = schirmer_annotation[!duplicated(schirmer_annotation), ]
schirmer_key$Diagnosis <- c(rep("Control", 9), rep("MS", 12))
schirmer_annotation$barcode <- gsub("(.*)_(.*)", "\\1", rownames(schirmer_annotation))
schirmer_key

Warning message:
“Removing 8211 cells missing data for vars requested”
Warning message:
“Removing 8211 cells missing data for vars requested”


,sample,Original_sample_name,pathology,Diagnosis
,<chr>,<chr>,<chr>,<chr>
AAACCTGTCAGTTGAC-1_SRR9123038,SRR9123038,C1,control_white_matter,Control
AAACCTGCATAACCTG-1_SRR9123040,SRR9123040,C2,control_white_matter,Control
AAAGCAACACTTGGAT-1_SRR9123039,SRR9123039,C3,control_white_matter,Control
AAACCTGCAGGTGGAT-1_SRR9123041,SRR9123041,C4,control_white_matter,Control
AAACGGGAGGGATCTG-1_SRR9123034,SRR9123034,C5,control_white_matter,Control
AAACGGGGTTACGTCA-1_SRR9123035,SRR9123035,C6,control_white_matter,Control
AAACCTGAGGGTATCG-1_SRR9123036,SRR9123036,C7,control_white_matter,Control
AACTCAGTCTTAACCT-1_SRR9123037,SRR9123037,C8,control_white_matter,Control
AACACGTTCGTGGTCG-1_SRR9123032,SRR9123032,C9,control_white_matter,Control


We'll save the metadata information for use in the Step 13 script.

In [6]:
saveRDS(schirmer_key, 'outputs/schirmer_key.rds')
saveRDS(schirmer_annotation, 'outputs/schirmer_annotation.rds')

Next we'll calculate the nuclear fractions. Note that this cell will take a long time to run.

Again, this code cell is commented out. If you have obtained the .bam and .bai files and wish to run DropletQC yourself, remove the "#" on each line in the below cell to un-comment the code. You can do this by selecting the lines and entering `Ctrl + /` or `Cmd + /` (depending on your platform OS).

In [7]:
#schirmer_nf_df <- c()
#
#for(x in unique(schirmer_key$sample)){
#
#    sample_ident = schirmer_key %>% filter(sample == x) %>% .$Original_sample_name
#
#    print(paste0("Processing sample ", sample_ident, "..."))
#    
#    schirmer_nf_tmp = nuclear_fraction_tags(bam = paste0('inputs/schirmer/CellRanger/', sample_ident, '_10x/outs/possorted_genome_bam.bam'), 
#                     bam_index = paste0('inputs/schirmer/CellRanger/', sample_ident, '_10x/outs/possorted_genome_bam.bam.bai'),
#                     barcodes = paste0('inputs/schirmer/CellRanger/', sample_ident, '_10x/outs/raw_feature_bc_matrix/barcodes.tsv.gz'))
#
#    rownames(schirmer_nf_tmp) <- paste0(rownames(schirmer_nf_tmp), '_', x)
#
#    if(x == 1){
#        schirmer_nf_df <- schirmer_nf_tmp
#    } else{
#        schirmer_nf_df <- rbind(schirmer_nf_df, schirmer_nf_tmp)
#    }
#}
#
#print(dim(schirmer_nf_df))
#
#saveRDS(schirmer_nf_df, 'outputs/schirmer_nuclear_fraction_df.rds')

Lastly, we'll save the combined dataset Seurat object and separated Absinta and Schirmer Seurat objects for easier use in the following scripts.

In [8]:
saveRDS(human, 'outputs/combined_human_seurat_object.rds')
saveRDS(absinta, 'outputs/absinta_seurat_object.rds')
saveRDS(schirmer, 'outputs/schirmer_seurat_object.rds')

In [9]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: macOS Big Sur ... 10.16

Matrix products: default
BLAS/LAPACK: /Users/liddelowlab/mambaforge/envs/EpiMemAstros/lib/libopenblasp-r0.3.28.dylib;  LAPACK version 3.12.0

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

time zone: America/New_York
tzcode source: system (macOS)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] Seurat_5.1.0         SeuratObject_5.0.2   sp_2.1-4            
[4] dplyr_1.1.4          ggplot2_3.5.1        data.table_1.15.4   
[7] DropletQC_0.0.0.9000

loaded via a namespace (and not attached):
  [1] deldir_2.0-4           pbapply_1.7-2          gridExtra_2.3         
  [4] rlang_1.1.4            magrittr_2.0.3         RcppAnnoy_0.0.22      
  [7] spatstat.geom_3.3-4    matrixStats_1.4.1      ggridges_0.5.6        
 [10] compiler_4.3.3         png_0.1-8              vctrs_

In [10]:
version

               _                           
platform       x86_64-apple-darwin13.4.0   
arch           x86_64                      
os             darwin13.4.0                
system         x86_64, darwin13.4.0        
status                                     
major          4                           
minor          3.3                         
year           2024                        
month          02                          
day            29                          
svn rev        86002                       
language       R                           
version.string R version 4.3.3 (2024-02-29)
nickname       Angel Food Cake             